# Mehrstufige Extraktion von Metadateninformationen aus sächsischen OER Inhalten

Sebastian Zug, André Dietrich 
TU Bergakademie Freiberg, Institut für Informatik

In [1]:
import pandas as pd

In [2]:
import sys
sys.path.append('../src')
from DataHandler import DataHandler

## 1. Schritt: Bewertung der Metadaten von OER Inhalten aus dem LMS OPAL

In [3]:
dataHandler = DataHandler("config.yaml")
df_opalmeta = dataHandler.load_data("data_files.raw_data.df_opalmeta")

📂 Lade: OPAL_files_attrib.p
   ✅ Geladen: 14,934 Zeilen × 12 Spalten
   ✅ Geladen: 14,934 Zeilen × 12 Spalten


In [4]:
df_opalmeta.shape

(14934, 12)

In [5]:
relevant_file_type = ['pptx', 'pdf', 'docx', 'xlsx']
df_opalmeta[df_opalmeta["pipe:file_type"].isin(relevant_file_type)].shape

(8658, 12)

In [6]:
def generateEmptinessStatistics(df, file_type_filter = False, base_count = 0):
    if file_type_filter:
        df = df[df["pipe:file_type"].isin(relevant_file_type)]
    df_empty = df.apply(lambda x: x == '').sum(axis=0).rename("count").to_frame()

    if base_count > 0:
        df_empty['empty_in_percent'] = df_empty['count'] / base_count * 100
    else:
        df_empty['empty_in_percent'] = df_empty / len(df) * 100
    return df_empty 

In [7]:
generateEmptinessStatistics(df_opalmeta)

,count,empty_in_percent
opal:filename,0,0.000000
opal:oer_permalink,0,0.000000
opal:license,0,0.000000
opal:author,14552,97.442078
opal:title,14508,97.147449
opal:comment,14698,98.419713
opal:language,14897,99.752243
opal:publicationMonth,14653,98.118388
opal:publicationYear,14653,98.118388
opal:revisedAuthor,14552,97.442078


In [8]:
df_opalmeta['pipe:file_type'].value_counts().head(20)

pipe:file_type
pdf      7766
jpg      1180
mkv       875
png       616
mp4       609
zip       509
docx      447
html      381
pptx      242
py        217
xlsx      203
ipynb     170
epub      108
m         101
mp3        93
pyc        82
mlx        76
pages      73
dotx       62
cpp        58
Name: count, dtype: int64

Ok, wer sind die metainformationsspezifischen Vorbilder?

In [9]:
df_opalmeta['opal:author'].value_counts().head(15)

opal:author
                                         14552
Frank Babick                               102
Prof. Dr.-Ing. Johann Zitzelsberger         76
Dominik Kern                                20
Monique Meier                               10
Daniela Dobeleit                             7
Benno Wessely; Frank Babick                  6
Valerie Uhlig                                5
Simon Razniewski                             4
Guido Philipp                                4
Nielsen Book Data                            4
Lea Christ                                   3
Hartmut Simmert                              3
Jessica Kluge und Melanie Trützschler        3
Vanessa Lange                                3
Name: count, dtype: int64

Welche Einträge davon sind aber tatsächliche Personen?

In [10]:
df_opal_checked = df_opalmeta[df_opalmeta["opal:author"] != ""].\
    loc[df_opalmeta["opal:revisedAuthor"].\
    apply(lambda x: isinstance(x, list) and len(x) == 0)]

In [11]:
df_opal_checked[["pipe:ID", "opal:author", "opal:revisedAuthor"]]

,pipe:ID,opal:author,opal:revisedAuthor
203,8T_kyL931uXU,tuuwi,[]
609,3qWrx7Jiwlww,ESRI,[]
1345,1I64MwnfFqi0,AMassessoriaPublicitaria,[]
1446,1Pkdn1EeAbO8,a,[]
4210,5hSnIFh2B99Y,Kompetenzteam Lernsysteme GU Frankfurt,[]
5874,103oakfHh1RQM,Soziopod,[]
7914,6nvurAD8bwkk,Cambio e.V,[]
10079,10FxaP3vJ4SKY,MMST_WiSE21_22_Gruppe 9,[]
12157,5KPZekiRpTh8,Nielsen Book Data,[]
12158,5261EgqHcKsg,Publishers Association,[]


In [12]:
print(f"{df_opalmeta[df_opalmeta["opal:author"] != ""].shape[0]} Namen und Institutionen sind enthalten, davon wurden {df_opal_checked.shape[0]} als nicht Personen identifiziert.")

382 Namen und Institutionen sind enthalten, davon wurden 18 als nicht Personen identifiziert.


## Schritt 2: Traditionelle Aggregation der Metadaten

Welche Dateitypen sind denn  in den OPAL OER Daten überhaupt vertreten?

Im Rahmen der ersten Untersuchungsreihe fokussieren wir uns auf {`pdf`, `docx`, `pptx`, `xlsx`}. Ausgangspunkt war eine Stichprobenprüfung der Mediendateien, die keinerlei Metadaten enthielten.

In [13]:
relevant_file_type = ['pdf', 'docx', 'pptx', 'xlsx', 'md']
print(df_opalmeta[df_opalmeta["pipe:file_type"].isin(relevant_file_type)]["pipe:file_type"].shape[0] )
print(df_opalmeta[df_opalmeta["pipe:file_type"].isin(relevant_file_type)]["pipe:file_type"].shape[0] / df_opalmeta.shape[0])

8666
0.5802865943484666


Offenbar decken wir mit dieser Auswahl etwa 58 Prozent der in OPAL enthaltenen Dokumente ab. Für diese Datentypen existieren unterschiedliche Metadateninformationen, die in die Dateien eingebettet sind. 

![](https://redmondmag.com/articles/2019/02/25/~/media/ECG/redmondmag/Images/2019/02/0225red_metadata1.ashx)

Ein Skript der CL Pipeline hat diese, sofern vorhanden, für uns ausgelesen ...

In [14]:
df_filesmeta = dataHandler.load_data("data_files.raw_data.df_filesmeta")

📂 Lade: OPAL_files_meta.p
   ✅ Geladen: 8,666 Zeilen × 10 Spalten


In [15]:
df_filesmeta.shape

(8666, 10)

In [16]:
generateEmptinessStatistics(df_filesmeta)

,count,empty_in_percent
pipe:ID,0,0.000000
pipe:file_type,0,0.000000
file:author,3096,35.725825
file:keywords,7653,88.310639
file:subject,7207,83.164090
file:title,3766,43.457189
file:created,0,0.000000
file:modified,0,0.000000
file:language,672,7.754443
file:revisedAuthor,3096,35.725825


Für 65 Prozent der Autoren `file:author` können wir zumindest einen Vorschlage zum Namen machen? Das klingt schon mal vielversprechend. Allerdings trübt die Anzahl von echten Namen `file:revisedAuthor` den Eindruck. Lassen Sie uns die Metadaten der verschiedenen Quellen vergleichen, um die Qualitäten zu prüfen.

In [17]:
df_merge = pd.merge(df_filesmeta, df_opalmeta.drop(['pipe:file_type'], axis=1), on="pipe:ID", how="left")

Schauen wir uns zunächst positiv Beispiel an, die aus beiden Quellen enthielten.

In [18]:
df_merge[(df_merge["opal:author"]!="") &  (df_merge["file:author"]!="")][["opal:author", "file:author"]].head(10)

,opal:author,file:author
151,Frank Babick,Frank Babick
155,Prof. Dr.-Ing. Johann Zitzelsberger,Prof. Dr.-Ing. Johann Zitzelsberger
177,Prof. Dr.-Ing. Johann Zitzelsberger,Prof. Dr.-Ing. Johann Zitzelsberger
360,ESRI,ESRI
368,Prof. Dr. Nadine Bergner,Prof. Dr. Nadine Bergner
418,Jost-Hinrich Eschenburg,Jost-Hinrich Eschenburg
437,"Roeder, Klimova, Kuhn",Institute of Medical Informatics and Biometry ...
456,Frank Babick,Frank Babick u.a.
479,Frank Babick,Frank Babick
480,Frank Babick,Frank Babick


Und nun umgekehrt, helfen uns die aus den Dateien extrahierten Metadaten bei der Identifikation der Autoren? Das Bild ist durchwachsen ...

In [19]:
df_merge[(df_merge["opal:author"]=="") &  (df_merge["file:author"]!="")][["opal:author", "file:author"]].head(10)

,opal:author,file:author
5,,Ralf Laue
6,,None
8,,"Walter, Susanne (FIN A 2.3)"
12,,home
13,,home
14,,home
23,,Anja
24,,Anna
25,,P. Menzel;Hamza
26,,lschlenker


Wir brauchen einen weiteren Mechanismus der uns bei der Extraktion unterstützt! 

### Schritt 3: KI basierte Metadatenaggregation

Die CL Pipeline wurde um ein weiteres Modul erweitert, dass aktuell ein llama3 LLM verwendet, um die Titel, Autoren und Keyworte zu extrahieren.

In [20]:
df_aimeta = dataHandler.load_data("data_files.raw_data.df_aimeta")
df_aimeta

📂 Lade: OPAL_ai_meta.p
   ✅ Geladen: 4,548 Zeilen × 13 Spalten


,pipe:ID,pipe:file_type,ai:author,ai:revisedAuthor,ai:affilation,ai:title,ai:type,ai:keywords_ext,ai:keywords_gen,ai:keywords_dnb,ai:dewey,ai:affiliation,ai:summary
0,8I6sM5zapD60,pdf,"Stephan Gerhold, Marcel Beyer",[Vorname='Stephan' Familienname='Gerhold' Tite...,,Übung 3 Photogrammetrie,Aufgabenblatt,"Passpunkte, digitale Entzerrung, Photogrammetr...","Photogrammetrie, digitale Entzerrung, Resampli...","Photogrammetrie, Bildverarbeitung, Digitale En...","[{'notation': '004', 'label': 'Informatik', 's...",,
1,8ZICOHBmAHyQ,pdf,Keine Angabe.,[],,3.1 Grundlagen der Tragwerke,Skript,"Tragwerke, Lagerreaktionen, Gleichgewichtsbedi...","Tragwerke, statische Bestimmtheit, Lagerreakti...","Statik, Tragwerke, Mechanik, Festigkeitslehre,...","[{'notation': '629', 'label': 'Andere Fachrich...",,
2,8Lfz8SAKa6k0,pdf,Robert Ringel,[Vorname='Robert' Familienname='Ringel' Titel=''],,Gedanken und Notizen zu unserem Mini-KI-Worksh...,Protokoll,"KI-Systeme, Automatisierung, Industrie, Arbeit...","Künstliche Intelligenz, Automatisierung, Indus...","Künstliche Intelligenz, Automatisierung, Indus...","[{'notation': '004', 'label': 'Informatik', 's...",,
3,3ztCv-WpxJ4U,pdf,Norbert Engemaier,[Vorname='Norbert' Familienname='Engemaier' Ti...,Technische Universität Dresden,Allgemeine Anforderungen,Aufgabenblatt,"Bibliographie, Literaturangaben, Sekundärquell...","Bibliographie, Literaturverzeichnis, wissensch...","Wissenschaftliches Arbeiten, akademische Schri...","[{'notation': '100', 'label': 'Philosophie und...",Technische Universität Dresden,Dieses Dokument beschreibt die Anforderungen u...
4,6mOhjfscZK2A,pdf,Keine Angabe.,[],Technische Universität Dresden,"Vorlesung Technische Mechanik I - AGBF, TU Dre...",Vorlesungsfolien,"Technische Mechanik, Starrer Körper, Einzelmom...","Technische Mechanik, Starrer Körper, Einzelmom...","Technische Mechanik, Starrer Körper, Kraft, Mo...","[{'notation': '531', 'label': 'Klassische Mech...",Technische Universität Dresden,Die Vorlesung Technische Mechanik I behandelt ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4543,8Qx2WQd_ANdI,pdf,"Dr. Sabina Tsapaeva, Prof. Dr. Alexander Lasch...",[Vorname='Sabina' Familienname='Tsapaeva' Tite...,,Grundlagen der Linguistik und Sprachgeschichte...,Aufgabenblatt,"Dialektologie, Variationslinguistik, Diaphasis...","Dialektologie, Variationslinguistik, Sprachges...","Linguistik, Deutsche Sprache, Dialektologie, V...","[{'notation': '410', 'label': 'Linguistik', 's...",,Die Grundlagen der Linguistik und Sprachgeschi...
4544,7rwytM7w-8HE,pdf,Keine Angabe.,[],,"Emotionsmanagement (Fähigkeit, eigene Emotione...",Skript,"Emotionsmanagement, Emotionswahrnehmung, Emoti...","Emotionsmanagement, akademische Intelligenz, W...","Intelligenz, Kreativität, Emotionsmanagement, ...","[{'notation': '150', 'label': 'Psychologie', '...",,Die Intelligenztheorien behandeln verschiedene...
4545,8MBQRpA2XSQU,pdf,Keine Angabe.,[],,Chaotisches Verhalten,Skript,"Chaostheorie, Deterministisches Chaos, Autokat...","Chaostheorie, Deterministisches Chaos, Autokat...","Chaostheorie, Deterministisches Chaos, Chemisc...","[{'notation': '510', 'label': 'Mathematik', 's...",,
4546,10MPMUvFFT_H0,pdf,"Marco Hamann, Michael Meinhold",[Vorname='Marco' Familienname='Hamann' Titel='...,Hochschule für Technik und Wirtschaft Dresden,Übungsblatt 9 Aufgaben mit Lösungshilfe,Aufgabenblatt,"Geometrie, Drehung, Parameterdarstellung, Matr...","Geometrie, Drehung, Matrixdarstellung, Rodrigu...","Geometrie, Drehung, Abbildung, Koordinatensyst...","[{'notation': '516', 'label': 'Geometrie', 'sc...",Hochschule für Technik und Wirtschaft Dresden,Dieses Übungsblatt zum Thema Geometrie enthält...


In [21]:
# Zähle die Einträge die leer sind oder leere Listen enthalten
def is_empty_or_empty_list(x):
    """Prüft ob ein Wert leer ist (leerer String oder leere Liste)"""
    if x == '':
        return True
    if isinstance(x, list) and len(x) == 0:
        return True
    return False

df_aimeta_empty = df_aimeta.apply(lambda x: x.apply(is_empty_or_empty_list)).sum(axis=0).rename("count").to_frame()
df_aimeta_empty['empty_in_percent'] = df_aimeta_empty['count'] / len(df_aimeta) * 100   
df_aimeta_empty.sort_values(by='empty_in_percent', ascending=False)

,count,empty_in_percent
ai:affilation,2769,60.883905
ai:affiliation,2431,53.452067
ai:revisedAuthor,1417,31.156552
ai:summary,1141,25.087951
ai:title,1,0.021988
pipe:file_type,0,0.000000
ai:author,0,0.000000
pipe:ID,0,0.000000
ai:type,0,0.000000
ai:keywords_gen,0,0.000000


In [22]:
df_aimeta['ai:type'].value_counts(normalize=True).head(20)

ai:type
Aufgabenblatt                  0.315963
Vorlesungsfolien               0.268690
Skript                         0.159850
Protokoll                      0.056948
Seminararbeit                  0.041117
Dokumentation                  0.035620
Tutorial                       0.028144
Präsentation                   0.020009
Paper                          0.016271
Materialhandreichung           0.009675
Buch                           0.007256
Bachelor- oder Masterarbeit    0.003078
Stoffverteilungsplan           0.002419
Sonstiges                      0.001979
Exposé                         0.001539
Praktikumsbericht              0.001539
Arbeitsblatt                   0.001099
Aufgabenpool                   0.001099
Poster                         0.001099
Handreichung                   0.000880
Name: proportion, dtype: float64

In [23]:
df_aimeta.shape[0] / df_filesmeta[df_filesmeta["pipe:file_type"].isin(relevant_file_type)].shape[0] 

0.5248096007385183

Für etwa 52 Prozent der Office-Dateien und pdfs konnten entsprechende Informationen erschlossen werden. Die Frage ist, wie schlüssig diese Resultate sind. Fusionieren wir also alle Datensätze miteinander, um das zu untersuchen.

In [24]:
df_merge_all = pd.merge(df_aimeta, df_merge.drop(['pipe:file_type'], axis=1), on="pipe:ID", how="left")
df_merge_all[df_merge_all["opal:author"].str.contains("Löwe", na=False, case=False)].iloc[0]

pipe:ID                                                       1PGOlNUd1m7g
pipe:file_type                                                        pptx
ai:author                                                      Oliver Löwe
ai:revisedAuthor           [Vorname='Oliver' Familienname='Löwe' Titel='']
ai:affilation                                     TU Bergakademie Freiberg
ai:title                 Zeichnungen bergbaulicher Anlagen (Leupoldsamm...
ai:type                                                       Präsentation
ai:keywords_ext          Montanwesen, Erzgebirge, Bergbaumuseum, Gruben...
ai:keywords_gen          Montanwesen, Bergbau, Erzgebirge, Montanhistor...
ai:keywords_dnb          Bergbau, Montanhistorie, Erzgebirge, Bergakade...
ai:dewey                 [{'notation': '930', 'label': 'Geschichte des ...
ai:affiliation                                    TU Bergakademie Freiberg
ai:summary               Die Leupoldsammlung umfasst historische Zeichn...
file:author              

Schauen wir uns also zunächst die Datensätze an, die sowohl Metadaten aus OPAL mitbrachten, für die aber auch Daten generiert werden konnten.

In [25]:
df_merge_all[ (df_merge_all["opal:author"]!="") &
             ((df_merge_all["file:author"]!="") | (df_merge_all["ai:author"]!=""))]\
             [["opal:author", "opal:revisedAuthor", "file:revisedAuthor", "ai:revisedAuthor"]]

,opal:author,opal:revisedAuthor,file:revisedAuthor,ai:revisedAuthor
100,Frank Babick,[Vorname='Frank' Familienname='Babick' Titel=''],[Vorname='Frank' Familienname='Babick' Titel=''],[Vorname='Frank' Familienname='Babick' Titel='']
118,Prof. Dr.-Ing. Johann Zitzelsberger,[Vorname='Johann' Familienname='Zitzelsberger'...,[Vorname='Johann' Familienname='Zitzelsberger'...,[Vorname='Johann' Familienname='Zitzelsberger'...
191,Jens-uwe Grabowski,[Vorname='Jens-uwe' Familienname='Grabowski' T...,,[Vorname='Ralf' Familienname='Romeike' Titel='']
256,Prof. Dr. Nadine Bergner,[Vorname='Nadine' Familienname='Bergner' Titel...,[Vorname='Nadine' Familienname='Bergner' Titel...,[Vorname='Nadine' Familienname='Bergner' Titel...
291,Jost-Hinrich Eschenburg,[Vorname='Jost-Hinrich' Familienname='Eschenbu...,[Vorname='Jost-Hinrich' Familienname='Eschenbu...,[Vorname='J.-H.' Familienname='Eschenburg' Tit...
...,...,...,...,...
4459,Prof. Dr.-Ing. Johann Zitzelsberger,[Vorname='Johann' Familienname='Zitzelsberger'...,[Vorname='Johann' Familienname='Zitzelsberger'...,[Vorname='Johann' Familienname='Zitzelsberger'...
4467,Prof. Dr.-Ing. Johann Zitzelsberger,[Vorname='Johann' Familienname='Zitzelsberger'...,[Vorname='Johann' Familienname='Zitzelsberger'...,[]
4505,Frank Babick,[Vorname='Frank' Familienname='Babick' Titel=''],[Vorname='Frank' Familienname='Babick' Titel=''],[Vorname='Frank' Familienname='Babick' Titel='']
4511,Prof. Dr.-ing. Johann Zitzelsberger,[Vorname='Johann' Familienname='Zitzelsberger'...,[Vorname='Johann' Familienname='Zitzelsberger'...,[]


... und umgekehrt? Wo können `ai` und `file` zum Beispiel Lücken bei den Autoren füllen? 

In [26]:
df_merge_all = df_merge_all[~df_merge_all['ai:author'].str.contains("Ich kann")]

In [27]:
df_merge_all[ (df_merge_all["opal:author"]=="") &
              ((df_merge_all["file:author"]!="") | 
               (df_merge_all["ai:author"]!=""))][["opal:author", "file:author", "ai:revisedAuthor"]].head(10)

,opal:author,file:author,ai:revisedAuthor
0,,,[Vorname='Stephan' Familienname='Gerhold' Tite...
1,,,[]
2,,,[Vorname='Robert' Familienname='Ringel' Titel='']
3,,,[Vorname='Norbert' Familienname='Engemaier' Ti...
4,,,[]
5,,Ralf Laue,[]
6,,,[Vorname='Birgit' Familienname='Brandt' Titel='']
7,,,[]
8,,,[]
9,,,[]
